## Bayesian Optimization

### 1. 基本概念
Bayesian Optimization是一种用于全局优化黑箱函数的方法。最基础的应用，就是根据有限的采样点信息寻找某个二维平面或高维函数的最优点。这里的“最优点”通常指函数的极大值或极小值，取决于问题是希望最大化收益还是最小化成本

#### 1.1. 定义目标函数
首先，需要明确优化的目标函数，也就是**黑箱函数**。它的输入可以是二维或多维坐标，输出为函数值。这类函数通常未知或计算代价高，无法直接求导或使用传统优化方法，因此需要借助代理模型和采样策略进行优化

#### 1.2. 建立代理模型
在黑箱函数未知的情况下，通过代理模型近似目标函数。常用模型是**高斯过程**，也可使用其他回归模型。代理模型不仅能预测未采样点的函数值，还能提供预测的不确定性，从而指导探索，避免陷入局部最优

#### 1.3. 选择采样点
在代理模型的指导下，下一步是**选择最有价值的采样点**，这一步由**采集函数**完成。采集函数会综合考虑预测值和不确定性，从而决定下一步探索的位置。常见方法包括：
##### 1.3.1. **期望改进**：计算候选点可能带来的收益改进，选择可能提升最大值的点
##### 1.3.2. **上置信界**：在考虑预测值的基础上加入不确定性，使得选择既有潜力又值得探索的点

#### 1.4. 迭代优化
每次采样后更新代理模型，并通过采集函数选择新的采样点。多次迭代后，优化逐步收敛到全局最优点，或者在有限预算下找到接近最优的解，实现利用已有信息与探索未知区域的平衡

> 小结：Bayesian Optimization 最基础的应用，就是根据采样点寻找平面函数的最优点，这些最优点通常是极大值或极小值

### 2. 学习感悟
结合之前整理的关于 Bayesian Optimization 的一些信息，我发现它非常适合黑箱场景。利用其在未知函数下仍能高效寻找最优点的特性，再联系模型训练时面对的黑箱问题，我能理解为什么大家会热衷于将贝叶斯优化应用于超参数调优和实验设计

个人体会是，Bayesian Optimization 提供了一种系统化的黑箱搜索方法，也让我在面对信息有限的情况时，会自然而然地想用贝叶斯去进行下一步选择的尝试

### 3. 进一步探索
结合我已经掌握的关于 Bayesian Optimization 的主体思想，在传统高维黑箱函数最优点求解的基础上，我尝试思考一个具体的应用场景进行模拟测试：将 Bayesian Optimization 应用到二维迷宫寻找出口的问题

在这个模拟中，迷宫用一个二维矩阵表示，其中 `1` 表示墙或围栏，`0` 表示可行走空间，`E` 表示出口。矩阵中设置了一层不均匀但互相连通的墙，出口嵌在其中，起始位置随机选择在围栏内部。我每轮在可移动范围内随机抽取`sample_size`个采样点 `*`，测量这些点到出口的曼哈顿距离，然后用 Bayesian Optimization 推断下一步可能更接近出口的位置，每次只能上下或左右移动一格。如果当前轮采样结果未提供足够的置信度进行移动，则保持原地不动，并在新一轮进行不重复的采样；当两次移动都符合合理性时，再随机选择一个方向移动。该过程持续进行，直到找到出口

具体实现上，我们使用采样点及其对应的曼哈顿距离来训练高斯过程（GP）模型，这样 GP 可以预测任意点到出口的曼哈顿距离，同时给出预测的方差。在每一步移动时，我们会在当前位置四周的可行邻居格中，选择预测距离最小且置信度足够的点作为下一步的位置。

这样，即便对迷宫结构一无所知，也能在有限的采样次数内逐步逼近出口。这个过程与寻找平面函数最优点的思路非常相似，但又包含一定的变通

### 4. 一些感悟
实践的时候，在有些出生点能够很快找到很明确的行径路线，但是有的时候到达一个区域，会开始左右脑互搏，反复来回横跳，我认为这和迷宫本身结构、单次采样点选取、置信度大小、每轮采样点数目都有一定的关系，理论上只要每轮采样点数目设置足够大，置信度足够严格就可以很好缓解这一问题，但是应该还会存在一些更好的解决方案能够处理这一困境。我认为其中值得考虑的可能包含如下几个点：
#### 4.1. 记忆性存储：保留部分历史走过的位置或者 GP 预测过的候选点信息，避免重复走到已经探索过但预测不可靠的点，可以用一个简单的访问次数矩阵或者哈希表记录
#### 4.2. 动态置信度阈值：可以根据局部信息量或历史探索状态动态调整，如果长时间无移动，降低阈值以允许探索；如果连续多轮预测可靠，适当提高阈值以提高决策精度

In [ ]:
import numpy as np
import random
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern


# ---------- 迷宫初始化 ----------
maze = np.array([
    [1,1,1,1,1,1,1],
    [1,0,0,1,0,0,1],
    [1,0,1,1,0,1,1],
    [1,0,0,0,0,0,1],
    [1,1,1,0,1,0,1],
    [1,0,0,0,1,0,1],
    [1,1,1,1,1,1,1]
])
end_pos = (5,5)
free_cells = [(i,j) for i in range(maze.shape[0]) 
                        for j in range(maze.shape[1]) if maze[i,j]==0 and (i,j)!=end_pos]
start_pos = random.choice(free_cells)


def print_maze(pos):
    for i in range(maze.shape[0]):
        row = ''
        for j in range(maze.shape[1]):
            if (i,j) == pos:
                row += 'S '
            elif (i,j) == end_pos:
                row += 'E '
            else:
                row += f'{maze[i,j]} '
        print(row)
    print()

def manhattan_dist(p1, p2):
    return abs(p1[0]-p2[0]) + abs(p1[1]-p2[1])

def get_valid_moves(pos):
    moves = []
    for d in [(-1,0),(1,0),(0,-1),(0,1)]:
        new_pos = (pos[0]+d[0], pos[1]+d[1])
        if maze[new_pos] != 1:
            moves.append(new_pos)
    return moves


print("初始迷宫：")
print_maze(start_pos)

current_pos = start_pos
observed_X = []
observed_y = []

# 这里分别是：最大轮次、置信度阈值、每轮采样点数、当前轮次
max_iterations = 100
confidence_threshold = 0.75
sample_size = 6
round_count = 0

while round_count < max_iterations:
    round_count += 1
    print(f"--- 第{round_count}轮采样 ---")
    
    possible_points = [(i,j) for i in range(maze.shape[0]) 
                               for j in range(maze.shape[1]) 
                               if maze[i,j] != 1 and (i,j)!=current_pos]
    
    sampled_points = []
    distances = []
    used_points = set()
    
    # 随机采样
    while len(sampled_points) < sample_size and possible_points:
        p = random.choice(possible_points)
        if p in used_points:
            continue
        used_points.add(p)
        sampled_points.append(p)
        d = manhattan_dist(p, end_pos)
        distances.append(d)
        print(f"采样点 {p}, 曼哈顿距离: {d}")
    
    # 添加到观测集合
    observed_X.extend(sampled_points)
    observed_y.extend(distances)
    
    # 构建 Gaussian Process
    X_train = np.array(observed_X)
    y_train = np.array(observed_y)
    kernel = Matern(nu=2.5)
    gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6)
    gp.fit(X_train, y_train)
    
    # 预测下一步可走方向
    candidates = get_valid_moves(current_pos)
    if not candidates:
        print("无可行移动，保持原地 (￣︿￣)")
        continue
    
    X_candidates = np.array(candidates)
    mu, sigma = gp.predict(X_candidates, return_std=True)
    
    # 选择预测距离最小且置信度满足的移动
    min_idx = np.argmin(mu)
    if sigma[min_idx] < confidence_threshold:
        next_pos = candidates[min_idx]
        current_pos = next_pos
        print("移动到:", current_pos)
        print_maze(current_pos)
        # 清空观测，进入下一轮
        observed_X = []
        observed_y = []

        if current_pos == end_pos:
            print("已到达终点 E，搜索结束 (≧▽≦)")
            break
    else:
        print("置信度不足，继续下一轮采样 (*•̀ᴗ•́*)و ̑̑")

if current_pos != end_pos: 
    print("我已经很努力了，我走不出去了 (｡•́︿•̀｡) ...")

### 5. 一些优化
针对先前提到的问题，我尝试对 plan1 进行实现，对访问次数增加惩罚机制，具体一些细节如下：
   1. 访问次数矩阵 `visit_count`
      建立与迷宫同样大小的矩阵，记录每个格子被访问的次数

   2. 移动决策中引入惩罚
      在 GP 预测下一步距离的均值 `mu` 时，对每个候选格子的访问次数加上一个惩罚项：
      ```python
      mu[idx] += penalty_factor * visit_count[pos]
      ```
      因此访问次数越多的格子，被选中的概率就越低，从而减少重复横跳现象

通过这个简单的设计，再次进行迷宫探索时，很少出现反复横跳的现象，也基本不会出现走不出去 QwQ 的结果。在当前的小矩阵迷宫中，该方法表现良好，但对于更大、更复杂的迷宫，或者三维场景，其效果就不得而知了

In [ ]:
import numpy as np
import random
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

# ---------- 迷宫初始化 ----------
maze = np.array([
    [1,1,1,1,1,1,1],
    [1,0,0,1,0,0,1],
    [1,0,1,1,0,1,1],
    [1,0,0,0,0,0,1],
    [1,1,1,0,1,0,1],
    [1,0,0,0,1,0,1],
    [1,1,1,1,1,1,1]
])
end_pos = (5,5)
free_cells = [(i,j) for i in range(maze.shape[0]) 
                        for j in range(maze.shape[1]) if maze[i,j]==0 and (i,j)!=end_pos]
start_pos = random.choice(free_cells)

def print_maze(pos):
    for i in range(maze.shape[0]):
        row = ''
        for j in range(maze.shape[1]):
            if (i,j) == pos:
                row += 'S '
            elif (i,j) == end_pos:
                row += 'E '
            else:
                row += f'{maze[i,j]} '
        print(row)
    print()

def manhattan_dist(p1, p2):
    return abs(p1[0]-p2[0]) + abs(p1[1]-p2[1])

def get_valid_moves(pos):
    moves = []
    for d in [(-1,0),(1,0),(0,-1),(0,1)]:
        new_pos = (pos[0]+d[0], pos[1]+d[1])
        if maze[new_pos] != 1:
            moves.append(new_pos)
    return moves

print("初始迷宫：")
print_maze(start_pos)

current_pos = start_pos
observed_X = []
observed_y = []

# 这里分别是：最大轮次、置信度阈值、每轮采样点数、当前轮次
max_iterations = 100
confidence_threshold = 0.75
sample_size = 6
round_count = 0

# 访问次数矩阵
visit_count = np.zeros_like(maze)

while round_count < max_iterations:
    round_count += 1
    print(f"--- 第{round_count}轮采样 ---")
    
    possible_points = [(i,j) for i in range(maze.shape[0]) 
                               for j in range(maze.shape[1]) 
                               if maze[i,j] != 1 and (i,j)!=current_pos]
    
    sampled_points = []
    distances = []
    used_points = set()
    
    # 随机采样
    while len(sampled_points) < sample_size and possible_points:
        p = random.choice(possible_points)
        if p in used_points:
            continue
        used_points.add(p)
        sampled_points.append(p)
        d = manhattan_dist(p, end_pos)
        distances.append(d)
        print(f"采样点 {p}, 曼哈顿距离: {d}")
    
    # 添加到观测集合
    observed_X.extend(sampled_points)
    observed_y.extend(distances)
    
    # 构建 Gaussian Process
    X_train = np.array(observed_X)
    y_train = np.array(observed_y)
    kernel = Matern(nu=2.5)
    gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6)
    gp.fit(X_train, y_train)
    
    # 预测下一步可走方向
    candidates = get_valid_moves(current_pos)
    if not candidates:
        print("无可行移动，保持原地 (￣︿￣)")
        continue
    
    X_candidates = np.array(candidates)
    mu, sigma = gp.predict(X_candidates, return_std=True)
    
    # 访问次数惩罚
    penalty_factor = 1.0
    for idx, pos in enumerate(candidates):
        mu[idx] += penalty_factor * visit_count[pos]
    
    # 选择预测距离最小且置信度满足的移动
    min_idx = np.argmin(mu)
    if sigma[min_idx] < confidence_threshold:
        next_pos = candidates[min_idx]
        current_pos = next_pos
        visit_count[current_pos] += 1
        print("移动到:", current_pos)
        print_maze(current_pos)
        # 清空观测，进入下一轮
        observed_X = []
        observed_y = []
        
        if current_pos == end_pos:
            print("已到达终点 E，搜索结束 (≧▽≦)")
            break
    else:
        print("置信度不足，继续下一轮采样 (*•̀ᴗ•́*)و ̑̑")

if current_pos != end_pos: 
    print("我已经很努力了，我走不出去了 (｡•́︿•̀｡) ...")
